# Collecting and Cleaning Twitter Usernames for Athletes

This notebook collects and prepares Twitter usernames for athletes across a number of sports. It involved a SPARQL endpoint for soccer players, merges data from multiple sports, and cleans the data to create a dataset of athlete usernames.

The final `athletes` dataset includes athlete information. 
- `name`: Full name of the athlete
- `username`: Athlete's Twitter username
- `sport`: Athlete's sport

In [5]:
%pip install unidecode SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

## Collect Soccer Player Usernames

In [7]:
# setup SPARQL endpoint and write query
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
query = """
SELECT ?item ?itemLabel ?twitter
WHERE 
{
  ?item wdt:P106 wd:Q937857. # Occupation: football player
  OPTIONAL{?item wdt:P2002 ?twitter}
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 1000
"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

In [8]:
# execute query and parse results
try:
    results = sparql.query().convert()
except Exception as e:
    print(f"Error executing query: {e}")
    results = None

Error executing query: HTTP Error 403: Forbidden


### Process Query Results

In [9]:
# process results if query was successful
if results:
    data = []
    # get relevant fields
    for result in results["results"]["bindings"]:
        item = result["item"]["value"]
        item_label = result.get("itemLabel", {}).get("value", None)
        twitter = result.get("twitter", {}).get("value", None)
        data.append({"item": item, "item_label": item_label, "twitter": twitter})

    # convert to dataframe and save to csv
    soccer_players = pd.DataFrame(data)
    soccer_players.drop(columns=['item'])
    soccer_players.to_csv("soccer_players_twitter.csv", index=False)

    # filter out rows with missing usernames
    soccer_players = soccer_players[soccer_players['twitter'].notna()]
    print(soccer_players.head())
else:
    print("No results.")

No results.


## Read and Merge Data

In [16]:
# load data
baseball_players = pd.read_csv('../../data/twitter_accounts/baseball_accounts.txt')
basketball_players = pd.read_csv('../../data/twitter_accounts/basketball_accounts.txt')
football_players = pd.read_csv('../../data/twitter_accounts/football_accounts.txt')
hockey_players = pd.read_csv('../../data/twitter_accounts/hockey_accounts.txt')
soccer_players = pd.read_csv('../../data/twitter_accounts/soccer_accounts.csv')

In [17]:
# view column names
print(f'baseball: {baseball_players.columns}\
        \nbasketball:{basketball_players.columns}\
        \nfootball: {football_players.columns}\
        \nhockey: {hockey_players.columns}\
        \nsoccer: {soccer_players.columns}')

baseball: Index(['name', 'username'], dtype='object')        
basketball:Index(['Rk', 'Player', 'Twitter'], dtype='object')        
football: Index(['name', 'username'], dtype='object')        
hockey: Index(['name', 'username'], dtype='object')        
soccer: Index(['item', 'item_label', 'twitter'], dtype='object')


## Data Cleaning

In [18]:
# drop unnecessary columns
basketball_players = basketball_players.drop(columns=['Rk'])
soccer_players = soccer_players.drop(columns=['item'])

# rename columns for consistency
basketball_players = basketball_players.rename(columns={'Player': 'name', 'Twitter': 'username'})
soccer_players = soccer_players.rename(columns={'item_label': 'name', 'twitter': 'username'})

In [19]:
# print updated columns
print(f'basketball:{basketball_players.columns}')
print(f'soccer: {soccer_players.columns}')

basketball:Index(['name', 'username'], dtype='object')
soccer: Index(['name', 'username'], dtype='object')


### Merge Data

In [20]:
# add indicators for each sport
baseball_players['sport'] = 'baseball'
basketball_players['sport'] = 'basketball'
football_players['sport'] = 'football'
hockey_players['sport'] = 'hockey'
soccer_players['sport'] = 'soccer'

In [21]:
# concatenate all data
athletes = pd.concat([baseball_players, basketball_players, football_players, hockey_players, soccer_players], ignore_index=True)
athletes

,name,username,sport
0,David Aardsma,@TheDA53,baseball
1,Henry Aaron,@HenryLouisAaron,baseball
2,Andrew Abbott,@andrewabbott33,baseball
3,Cory Abbott,@Cabbott40,baseball
4,Jim Abbott,@jabbottum31,baseball
...,...,...,...
8530,Jan Sobol,NaN,soccer
8531,Antonio Salazar,NaN,soccer
8532,Ricardo van Rhijn,NaN,soccer
8533,Ian Holloway,NaN,soccer


### Drop Missing Values

In [22]:
# identify rows with missing usernames
athletes[athletes['username'].isna()]

,name,username,sport
3085,Bubbles Hawkins,NaN,basketball
3086,Mo Howard,NaN,basketball
3090,Dermie O'Connell,NaN,basketball
3092,Tal Skinner,NaN,basketball
3094,Bobby Watson,NaN,basketball
...,...,...,...
8530,Jan Sobol,NaN,soccer
8531,Antonio Salazar,NaN,soccer
8532,Ricardo van Rhijn,NaN,soccer
8533,Ian Holloway,NaN,soccer


In [23]:
# drop rows with no username
athletes = athletes.dropna(subset=['username'])

# verify no null values
athletes.isna().sum()

name        0
username    0
sport       0
dtype: int64

### Ensure username consistency

In [24]:
# remove @ from all usernames
athletes['username'] = athletes['username'].str.replace('@', '')

athletes

/var/folders/pc/p12_c8d534vd9y9j8r2jrzzw0000gn/T/ipykernel_28761/2470902036.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes['username'] = athletes['username'].str.replace('@', '')


,name,username,sport
0,David Aardsma,TheDA53,baseball
1,Henry Aaron,HenryLouisAaron,baseball
2,Andrew Abbott,andrewabbott33,baseball
3,Cory Abbott,Cabbott40,baseball
4,Jim Abbott,jabbottum31,baseball
...,...,...,...
8501,Wesley Sneijder,sneijder101010,soccer
8518,Carlos Cuéllar,Cuellar24,soccer
8520,Simon Mignolet,SMignolet,soccer
8525,Winston Bogarde,WinstonBogarde5,soccer


## Final Cleaned DataFrame

In [26]:
athletes.to_csv('../../data/accounts_final.csv', index=False)